In [71]:
import os
import sys
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering
from display_value import display_unique_categorical_values


# Spécifiez les chemins des quatre parties du CSV découpé
partie1 = "../Dataset/SBAnational_part_part1.csv"
partie2 = "../Dataset/SBAnational_part_part2.csv"
partie3 = "../Dataset/SBAnational_part_part3.csv"
partie4 = "../Dataset/SBAnational_part_part4.csv"

# Utilisez la fonction charger_csv pour obtenir un DataFrame avec toutes les données réassemblées
df = charger_csv(partie1, partie2, partie3, partie4)

/tmp/ipykernel_25193/4113109593.py:26: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  df = charger_csv(partie1, partie2, partie3, partie4)


In [72]:
df.shape

(899164, 27)

In [73]:
# Utiliser la fonction pour générer la description de votre DataFrame
generate_description(df)


,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[1862155010, 1060795001, 2197766005, 288116400..."
1,Name,object,14,0.00,779583,"[Hubbell Enterprises, LLC, MID WAY EXPRESS INC..."
2,City,object,30,0.00,32581,"[FORT LAUDERDALE, SAN MARCOS, RICHMOND, Newark..."
3,State,object,14,0.00,51,"[CA, NY, AR, TX]"
4,Zip,int64,0,0.00,33611,"[90813, 64119, 84107, 1757, 22033]"
5,Bank,object,1559,0.17,5802,"[BRANCH BK. & TR CO, LOUISIANA CAP. CERT. DEVE..."
6,BankState,object,1566,0.17,56,"[NY, MN, NC, AR, NJ]"
7,NAICS,int64,0,0.00,1312,"[441222, 423120, 812320, 323119, 624410]"
8,ApprovalDate,object,0,0.00,9859,"[1-Apr-10, 8-Sep-06, 20-Jul-05, 14-Jan-91, 26-..."
9,ApprovalFY,object,0,0.00,71,"[2005, 1995, 1998, 2002]"


In [74]:
df = df[df.NAICS != 0]
df.shape

(697216, 27)

In [75]:
# Filter rows based on conditions
missing_mis_status = df[(df['MIS_Status'].isna() & df['ChgOffDate'].notna()) | df['MIS_Status'].isna() & ((df['ChgOffPrinGr'].notna()) & df['ChgOffPrinGr'] > 0.0)]

# Print relevant columns
print(missing_mis_status[['MIS_Status', 'ChgOffDate', 'ChgOffPrinGr']])

# Update 'MIS_Status' for filtered rows
df.loc[missing_mis_status.index, 'MIS_Status'] = 'CHGOFF'

       MIS_Status ChgOffDate ChgOffPrinGr
833           NaN        NaN       $0.00 
897           NaN        NaN       $0.00 
1051          NaN        NaN       $0.00 
1115          NaN        NaN       $0.00 
3005          NaN        NaN       $0.00 
...           ...        ...          ...
875154        NaN        NaN       $0.00 
875227        NaN        NaN       $0.00 
875441        NaN        NaN       $0.00 
885904        NaN        NaN       $0.00 
898182        NaN        NaN       $0.00 

[1716 rows x 3 columns]


In [76]:
print(df.loc[missing_mis_status.index, 'MIS_Status'])

833       CHGOFF
897       CHGOFF
1051      CHGOFF
1115      CHGOFF
3005      CHGOFF
           ...  
875154    CHGOFF
875227    CHGOFF
875441    CHGOFF
885904    CHGOFF
898182    CHGOFF
Name: MIS_Status, Length: 1716, dtype: object


In [77]:
missing_bank_s = df[(df['BankState'].isna() & df['Bank'].isna())]

print(missing_bank_s[['BankState', 'Bank', 'City', 'State', 'MIS_Status']])

       BankState Bank            City State MIS_Status
16108        NaN  NaN          DALLAS    TX      P I F
33663        NaN  NaN   RAQUETTE LAKE    NY      P I F
34582        NaN  NaN    OLMSTEDVILLE    NY      P I F
39732        NaN  NaN    OLMSTEDVILLE    NY      P I F
44475        NaN  NaN     WEST COVINA    CA      P I F
...          ...  ...             ...   ...        ...
866442       NaN  NaN          LOXLEY    AL      P I F
866444       NaN  NaN      FORT WORTH    TX      P I F
876947       NaN  NaN   MECHANICSBURG    PA      P I F
877019       NaN  NaN  MARINA DEL REY    CA      P I F
897474       NaN  NaN      FORT WAYNE    IN      P I F

[622 rows x 5 columns]


In [78]:
# Compter les occurrences de chaque catégorie dans la colonne 'MIS_Status'
mis_status_counts = missing_bank_s['MIS_Status'].value_counts()
print(mis_status_counts)


P I F     614
CHGOFF      8
Name: MIS_Status, dtype: int64


In [79]:
df = df.drop(missing_bank_s.index, axis=0)
df.shape

(696594, 27)

In [80]:
# Remplacer les valeurs manquantes dans 'City' par la valeur la plus fréquente pour cet État
df['City'] = df.groupby('State')['City'].transform(lambda x: x.fillna(x.mode()[0]))

# Remplacer les valeurs manquantes dans 'State' par la valeur la plus fréquente pour cette ville
df['State'] = df.groupby('City')['State'].transform(lambda x: x.fillna(x.mode()[0]))

# Supprimer les lignes avec des valeurs manquantes dans 'City' et 'State'
df.dropna(subset=['City', 'State'], inplace=True)

# Afficher les dimensions du DataFrame après le traitement
print("Nombre de lignes après le traitement des valeurs manquantes dans 'City' et 'State':", len(df))

Nombre de lignes après le traitement des valeurs manquantes dans 'City' et 'State': 696586


In [81]:
# Sélectionner les observations avec BankState manquant et Bank non manquant
missing_Bank_State = df[(df['BankState'].isna() & df['Bank'].notna())]

# Afficher les valeurs de BankState et Bank pour les observations concernées
print(missing_Bank_State[['BankState', 'Bank']])

# Modifier les valeurs de la colonne BankState à 'HS'
df.loc[missing_Bank_State.index, 'BankState'] = 'HS'

       BankState                            Bank
145106       NaN             KOREA EXCHANGE BANK
424311       NaN  STANDARD CHARTERED BK KOREA LT
859233       NaN  THE BANK OF EAST ASIA, LIMITED


In [82]:
# Afficher les valeurs de BankState et Bank pour les observations concernées après la modification
print(df.loc[missing_Bank_State.index, ['BankState', 'Bank']])

       BankState                            Bank
145106        HS             KOREA EXCHANGE BANK
424311        HS  STANDARD CHARTERED BK KOREA LT
859233        HS  THE BANK OF EAST ASIA, LIMITED


In [83]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]
df.NewExist.value_counts()

1.0    502343
2.0    193378
Name: NewExist, dtype: int64

In [84]:
df['RevLineCr'].value_counts()

N    269401
0    214171
Y    195125
T     14648
1        14
R        14
`         5
2         5
C         2
,         1
7         1
A         1
-         1
Q         1
Name: RevLineCr, dtype: int64

In [85]:
mapping = {'0': 'N', '1': 'Y'}

# Remplacer les valeurs selon le mapping défini
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

# Supprimer les valeurs qui ne sont pas 'N', 'Y', ou 'T'
valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

# Afficher les nouvelles valeurs uniques pour vérification
print(df['RevLineCr'].value_counts())

N    483572
Y    195139
Name: RevLineCr, dtype: int64


In [86]:
df['LowDoc'].value_counts()

N    607685
Y     66165
C       614
S       599
0       552
A       493
R        74
1         1
Name: LowDoc, dtype: int64

In [87]:
mapping = {'0': 'N', '1': 'Y'}
# Remplacer les valeurs selon le mapping défini
df['LowDoc'] = df['LowDoc'].replace(mapping)

# Supprimer les valeurs qui ne sont pas 'N', 'Y'
valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

# Afficher les nouvelles valeurs uniques pour vérification
print(df['LowDoc'].value_counts())

N    608237
Y     66166
Name: LowDoc, dtype: int64


In [88]:
# Appliquer la fonction currency_cleaning aux colonnes 'GrAppv' et 'SBA_Appv'
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)

In [89]:
# Filtrer les lignes où NAICS est égal à 0
df = df[df['NAICS'] != 0]

# Appliquer la fonction de reclassification des NAICS
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [90]:
NAICS = pd.get_dummies(df['NAICS'], prefix='NAICS', drop_first=False)
RevLineCr = pd.get_dummies(df['RevLineCr'], drop_first=True)
LowDoc = pd.get_dummies(df['LowDoc'], drop_first=True)

In [91]:
df.drop(['NAICS', 'RevLineCr', 'LowDoc'], axis=1, inplace=True)
df['RevLineCr'] = RevLineCr['Y']
df['LowDoc'] = LowDoc['Y']
df = pd.concat([df,NAICS], axis=1)

In [92]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df,MIS_Status], axis=1)
df = df.rename(columns={'P I F':'Approve'})

In [93]:
df

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,ApprovalFY,Term,...,NAICS_54,NAICS_55,NAICS_56,NAICS_61,NAICS_62,NAICS_71,NAICS_72,NAICS_81,NAICS_92,Approve
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,28-Feb-97,1997,84,...,0,0,0,0,0,0,0,0,0,1
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,28-Feb-97,1997,60,...,0,0,0,0,0,0,1,0,0,1
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,28-Feb-97,1997,180,...,0,0,0,0,1,0,0,0,0,1
5,1000084002,"B&T SCREW MACHINE COMPANY, INC",PLAINVILLE,CT,6062,"TD BANK, NATIONAL ASSOCIATION",DE,28-Feb-97,1997,120,...,0,0,0,0,0,0,0,0,0,1
7,1000094005,WEAVER PRODUCTS,SUMMERFIELD,FL,34491,REGIONS BANK,AL,28-Feb-97,1997,84,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899156,9995473009,"FABRICATORS STEEL, INC.",BALTIMORE,MD,21224,BANK OF AMERICA NATL ASSOC,MD,27-Feb-97,1997,60,...,0,0,0,0,0,0,0,0,0,1
899157,9995493004,PULLTARPS MFG.,EL CAJON,CA,92020,U.S. BANK NATIONAL ASSOCIATION,CA,27-Feb-97,1997,36,...,0,0,0,0,0,0,0,0,0,1
899159,9995573004,FABRIC FARMS,UPPER ARLINGTON,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,27-Feb-97,1997,60,...,0,0,0,0,0,0,0,0,0,1
899160,9995603000,FABRIC FARMS,COLUMBUS,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,27-Feb-97,1997,60,...,0,0,0,0,0,0,0,0,0,1


In [94]:
duplicates = df[df.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [LoanNr_ChkDgt, Name, City, State, Zip, Bank, BankState, ApprovalDate, ApprovalFY, Term, NoEmp, NewExist, CreateJob, RetainedJob, FranchiseCode, UrbanRural, ChgOffDate, DisbursementDate, DisbursementGross, BalanceGross, ChgOffPrinGr, GrAppv, SBA_Appv, RevLineCr, LowDoc, NAICS_11, NAICS_21, NAICS_22, NAICS_23, NAICS_31-33, NAICS_42, NAICS_44-45, NAICS_48-49, NAICS_51, NAICS_52, NAICS_53, NAICS_54, NAICS_55, NAICS_56, NAICS_61, NAICS_62, NAICS_71, NAICS_72, NAICS_81, NAICS_92, Approve]
Index: []

[0 rows x 46 columns]


In [95]:
df.isna().sum()

LoanNr_ChkDgt             0
Name                     10
City                      0
State                     0
Zip                       0
Bank                      0
BankState                 0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                  0
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
ChgOffDate           537529
DisbursementDate       1841
DisbursementGross         0
BalanceGross              0
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
RevLineCr                 0
LowDoc                    0
NAICS_11                  0
NAICS_21                  0
NAICS_22                  0
NAICS_23                  0
NAICS_31-33               0
NAICS_42                  0
NAICS_44-45               0
NAICS_48-49               0
NAICS_51                  0
NAICS_52                  0
NAICS_53            

In [96]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.shape

(674403, 39)

In [97]:
# Convertir le DataFrame en fichier CSV
df_to_csv(df, folder_path, 'SBA_Cleaned_maybe.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_maybe.csv


In [98]:
maybe_leaking_feature = ['SBA_Appv', 'GrAppv', 'RetainedJob', 'CreateJob']
df = df.drop(maybe_leaking_feature, axis=1)
df.shape

(674403, 35)

In [99]:
# Convertir le DataFrame en fichier CSV
df_to_csv(df, folder_path, 'SBA_Cleaned.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned.csv


In [100]:
y = df['Approve']
X = df.drop('Approve', axis=1)

In [101]:
df.shape

(674403, 35)

In [104]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score  # Correction de l'importation
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train = X_train.astype(str)


numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]


categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse=False)
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("nums", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ]
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(max_depth=25, min_samples_leaf=80, min_samples_split=3, n_estimators=70, max_samples=0.7, random_state=42))
])

set_config(transform_output="pandas")

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy:.3f}')

/home/utilisateur/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


MemoryError: Unable to allocate 154. GiB for an array with shape (539522, 38412) and data type float64